# Load Dataset


In [4]:
import os
import cv2
import numpy as np

# def load_images_from_folder(folder): 
#     images = [] 
#     for filename in os.listdir(folder): 
#         img = cv2.imread(os.path.join(folder, filename))  # Add os.path.join
#         if img is not None: 
#             images.append(img) 
#     return images
    
dirname = '/kaggle/input/brain-tumors-dataset/Data/'
normal_image_path = os.path.join(dirname, 'Normal')
glioma_tumor_path = os.path.join(dirname, 'Tumor', 'glioma_tumor')
meningioma_tumor_path = os.path.join(dirname, 'Tumor', 'meningioma_tumor')
pituitary_tumor_path = os.path.join(dirname, 'Tumor', 'pituitary_tumor')

# normal_images = load_images_from_folder(normal_image_path)
# glioma_images = load_images_from_folder(glioma_tumour_path) 
# meningioma_images = load_images_from_folder(meningioma_tumour_path) 
# pituitary_images = load_images_from_folder(pituitary_tumor_path)


# Resize  and Standardize Images into standard size

In [ ]:
# Define a function to resize images
def resize_images(folder):
    resized_images = []
  
    for filename in os.listdir(folder): 
        img = cv2.imread(os.path.join(folder, filename)) 
        resized_img = cv2.resize(img, (224, 224)) 
        resized_img = np.array(resized_img)
        resized_img = resized_img / 255.0
        resized_images.append(resized_img)
    return resized_images

# Resize normal and tumor images
normal_images = resize_images(normal_image_path)
glioma_images = resize_images(glioma_tumor_path)
meningioma_images = resize_images(meningioma_tumor_path)
pituitary_images = resize_images(pituitary_tumor_path)

# Training and Test Data Division

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Create labels for the images: 0 for normal, 1 for glioma, 2 for meningioma, 3 for pituitary
normal_label = np.zeros(normal_images.shape[0], dtype=int)
glioma_label = np.ones(glioma_images.shape[0], dtype=int)
meningioma_label = np.full(meningioma_images.shape[0], fill_value=2, dtype=int)
pituatary_label = np.full(pituatary_images.shape[0], fill_value=3, dtype=int)

# Concatenate the labels
X = np.concatenate((normal_images, glioma_images, meningioma_images, pituatary_images), axis=0)
y = np.concatenate((normal_label, glioma_label, meningioma_label, pituatary_label), axis=0)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Modelling

In [ ]:
!pip install tflearn

In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=17, activation="softmax"))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation

In [6]:
from keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator object
datagen = ImageDataGenerator(
    rotation_range=20,     # Range (0-180) within which to randomly rotate pictures
    width_shift_range=0.2, # Fraction of total width to randomly translate images
    height_shift_range=0.2,# Fraction of total height to randomly translate images
    zoom_range=0.2,        # Range for random zoom
    horizontal_flip=True,  # Randomly flip inputs horizontally
    vertical_flip=True)    # Randomly flip inputs vertically

# Fit the ImageDataGenerator object to your data
datagen.fit(X_train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

NameError: name 'X_train' is not defined

# Training The Model

In [ ]:
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1)
model.fit(X_train, y_train,verbose=1, batch_size=24, epochs=1000, validation_split=0.25, shuffle=True, callbacks=[es])

# Testing The Model

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
# Convert probabilities to class labels
y_pred = (y_pred > 0.5).astype(int)

# Tabulating Model SCore

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def find_score(y_test,y_pred):
    # Compute metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)

    # Print metrics
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print(f'ROC AUC: {roc_auc}')

find_score(y_test,y_pred)

# Save Model

In [ ]:
import pickle
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)


# Load Model

In [ ]:
with open('model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)



In [ ]:
# Define a function to preprocess new input data
def preprocess_input(image_path):
    # Load and resize the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (100, 100))
    
    # Normalize the image
    img = img / 255.0
    
    # Reshape the image
    img = img.reshape(1, 100, 100, 3)
    
    return img

# Define a function to make predictions on new input data
def predict(image_path):
    # Preprocess the input
    img = preprocess_input(image_path)
    
    # Make a prediction
    prediction = model.predict(img)
    
    # Convert the prediction to a class label
    label = (prediction > 0.5).astype(int)
    
    return label

demo='/kaggle/input/brain-tumors-dataset/Data/Normal/image1.jpg'
print(predict(demo))